In [14]:
import tensorflow as tf
import json
import pandas as pd
import base64
import requests

data = pd.read_csv('data\spamham.csv')


In [15]:
data

,Text,Class
0,Supply Quality China's EXCLUSIVE dimensions at...,1
1,over. SidLet me know. Thx.,0
2,"Dear Friend,Greetings to you.I wish to accost ...",1
3,MR. CHEUNG PUIHANG SENG BANK LTD.DES VOEUX RD....,1
4,Not a surprising assessment from Embassy.,0
...,...,...
4840,It has been prepared- ready for your signature...,0
4841,And to you too,0
4842,<html><head><style>P{margin:0px;padding:0px}body,1
4843,Robinson talking now voice cracking but resolu...,0


In [17]:
data['Text']

0       Supply Quality China's EXCLUSIVE dimensions at...
1                              over. SidLet me know. Thx.
2       Dear Friend,Greetings to you.I wish to accost ...
3       MR. CHEUNG PUIHANG SENG BANK LTD.DES VOEUX RD....
4               Not a surprising assessment from Embassy.
                              ...                        
4840    It has been prepared- ready for your signature...
4841                                       And to you too
4842     <html><head><style>P{margin:0px;padding:0px}body
4843    Robinson talking now voice cracking but resolu...
4844    PIR <preines(Wednesday February 10 2010 1:16 P...
Name: Text, Length: 4845, dtype: object

In [18]:

text_test = data['Text'][1]

text_test

'over. SidLet me know. Thx.'

In [20]:
def prepare_json(text):
    feature_spec = {
        "Text": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")])),
    }
    
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })

json_data = prepare_json(text_test)
    
endpoint = "http://103.190.215.173:8501/v1/models/spam-prediction-model:predict"
response = requests.post(endpoint, data=json_data)

prediction = response.json().get("predictions")
if prediction:
    prediction_value = prediction[0][0]
    result = "Spam" if prediction_value > 0.6 else "Bukan spam"
else:
    result = "Error: No predictions found in response."

print(result)

Bukan spam
